In [ ]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table = pd.read_html(url)
sp500_df = table[0]
sp500_tickers = sp500_df['Symbol'].tolist()


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")


start = '2010-01-01'
end = '2024-12-31'
window_size = 100
split_ratio = 0.7
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
table = pd.read_html(url)
sp500_df = table[0]
tickers = sp500_df['Symbol'].tolist()

for ticker in tickers:
    print(f"\n📈 Downloading data for {ticker}...")
    df = yf.download(ticker, start=start, end=end)

    if df.empty or 'Close' not in df.columns or len(df) < window_size + 1:
        print(f"Skipping {ticker} due to insufficient data.")
        continue

    df.sort_index(inplace=True)
    close_prices = df['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(close_prices)

    X, y = [], []
    for i in range(window_size, len(scaled_data)):
        X.append(scaled_data[i - window_size:i, 0])
        y.append(scaled_data[i, 0])

    X = np.array(X)
    y = np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    train_size = int(len(X) * split_ratio)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    print(f"Training shape for {ticker} - X_train: {X_train.shape}, X_test: {X_test.shape}")

    model = Sequential([
        GRU(units=50, return_sequences=True, input_shape=(window_size, 1)),
        Dropout(0.2),
        GRU(units=50),
        Dropout(0.2),
        Dense(units=1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        epochs=10,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        verbose=1
    )

    model_path = f"{ticker}_gru_model.h5"
    model.save(model_path)
    print(f"✅ Saved model to {model_path}")

print("\n✅ All models trained and saved.")


[*********************100%***********************]  1 of 1 completed


📈 Downloading data for MMM...
Training shape for MMM - X_train: (2571, 100, 1), X_test: (1102, 100, 1)


Epoch 1/10
81/81 [==============================] - 7s 64ms/step - loss: 0.0144 - val_loss: 4.8695e-04
Epoch 2/10
81/81 [==============================] - 6s 69ms/step - loss: 0.0028 - val_loss: 5.1909e-04
Epoch 3/10
81/81 [==============================] - 7s 90ms/step - loss: 0.0023 - val_loss: 4.4144e-04
Epoch 4/10
81/81 [==============================] - 8s 104ms/step - loss: 0.0022 - val_loss: 3.6399e-04
Epoch 5/10
81/81 [==============================] - 6s 76ms/step - loss: 0.0019 - val_loss: 3.5791e-04
Epoch 6/10
81/81 [==============================] - 5s 67ms/step - loss: 0.0018 - val_loss: 6.7120e-04
Epoch 7/10
81/81 [==============================] - 5s 64ms/step - loss: 0.0017 - val_loss: 4.7773e-04
Epoch 8/10
81/81 [==============================] - 5s 64ms/step - loss: 0.0017 - val_loss: 4.4571e-04
Epoch 9/10
81/81 [==============================] - 5s 64ms/step - loss: 0.0015 - val_loss: 3.4570e-04
Epoch 10/10
81/81 [==============================] - 5s 64ms/step - loss

[*********************100%***********************]  1 of 1 completed


Training shape for AOS - X_train: (2571, 100, 1), X_test: (1102, 100, 1)
Epoch 1/10
81/81 [==============================] - 14s 132ms/step - loss: 0.0061 - val_loss: 7.2041e-04
Epoch 2/10
81/81 [==============================] - 7s 87ms/step - loss: 0.0012 - val_loss: 6.4049e-04
Epoch 3/10
81/81 [==============================] - 8s 95ms/step - loss: 0.0011 - val_loss: 5.8686e-04
Epoch 4/10
81/81 [==============================] - 10s 122ms/step - loss: 9.6978e-04 - val_loss: 6.2629e-04
Epoch 5/10
81/81 [==============================] - 9s 116ms/step - loss: 9.6381e-04 - val_loss: 6.5581e-04
Epoch 6/10
81/81 [==============================] - 8s 97ms/step - loss: 8.5112e-04 - val_loss: 6.3406e-04
Epoch 7/10
81/81 [==============================] - 8s 94ms/step - loss: 7.9344e-04 - val_loss: 5.7636e-04
Epoch 8/10
81/81 [==============================] - 6s 76ms/step - loss: 7.6191e-04 - val_loss: 7.7957e-04
Epoch 9/10
81/81 [==============================] - 6s 71ms/step - loss: 7.464

...

Training shape for PG - X_train: (2571, 100, 1), X_test: (1102, 100, 1)
Epoch 1/10
81/81 [==============================] - 13s 93ms/step - loss: 0.0040 - val_loss: 6.1657e-04
Epoch 2/10
81/81 [==============================] - 6s 78ms/step - loss: 6.2438e-04 - val_loss: 2.7854e-04
Epoch 3/10
81/81 [==============================] - 6s 78ms/step - loss: 5.1021e-04 - val_loss: 3.6917e-04
Epoch 4/10
81/81 [==============================] - 6s 75ms/step - loss: 5.1022e-04 - val_loss: 5.2593e-04
Epoch 5/10
81/81 [==============================] - 7s 86ms/step - loss: 4.4359e-04 - val_loss: 0.0014
Epoch 6/10
81/81 [==============================] - 8s 99ms/step - loss: 4.6635e-04 - val_loss: 4.9779e-04
Epoch 7/10
81/81 [==============================] - 8s 103ms/step - loss: 4.3132e-04 - val_loss: 5.9080e-04
Epoch 8/10
81/81 [==============================] - 9s 112ms/step - loss: 3.6177e-04 - val_loss: 9.7767e-04
Epoch 9/10
81/81 [==============================] - 8s 96ms/step - loss: 3.34

[*********************100%***********************]  1 of 1 completed


Training shape for PGR - X_train: (2571, 100, 1), X_test: (1102, 100, 1)
Epoch 1/10
81/81 [==============================] - 13s 92ms/step - loss: 8.0032e-04 - val_loss: 5.0785e-04
Epoch 2/10
81/81 [==============================] - 6s 78ms/step - loss: 1.6300e-04 - val_loss: 4.0695e-04
Epoch 3/10
81/81 [==============================] - 6s 76ms/step - loss: 1.4335e-04 - val_loss: 2.9905e-04
Epoch 4/10
81/81 [==============================] - 6s 77ms/step - loss: 1.2680e-04 - val_loss: 1.7220e-04
Epoch 5/10
81/81 [==============================] - 6s 76ms/step - loss: 1.1287e-04 - val_loss: 2.4332e-04
Epoch 6/10
81/81 [==============================] - 6s 77ms/step - loss: 1.0962e-04 - val_loss: 1.6029e-04
Epoch 7/10
81/81 [==============================] - 6s 76ms/step - loss: 1.0171e-04 - val_loss: 2.2522e-04
Epoch 8/10
81/81 [==============================] - 7s 90ms/step - loss: 8.6980e-05 - val_loss: 1.4733e-04
Epoch 9/10
81/81 [==============================] - 8s 95ms/step - los

[*********************100%***********************]  1 of 1 completed


Training shape for PLD - X_train: (2571, 100, 1), X_test: (1102, 100, 1)
Epoch 1/10
81/81 [==============================] - 13s 97ms/step - loss: 0.0033 - val_loss: 8.6545e-04
Epoch 2/10
81/81 [==============================] - 6s 80ms/step - loss: 5.5603e-04 - val_loss: 5.0574e-04
Epoch 3/10
81/81 [==============================] - 6s 80ms/step - loss: 4.6326e-04 - val_loss: 4.1748e-04
Epoch 4/10
81/81 [==============================] - 7s 81ms/step - loss: 4.2644e-04 - val_loss: 5.6167e-04
Epoch 5/10
81/81 [==============================] - 6s 79ms/step - loss: 3.9535e-04 - val_loss: 7.3669e-04
Epoch 6/10
81/81 [==============================] - 6s 80ms/step - loss: 3.5378e-04 - val_loss: 5.5556e-04
Epoch 7/10
81/81 [==============================] - 7s 85ms/step - loss: 3.5169e-04 - val_loss: 8.0171e-04
Epoch 8/10
81/81 [==============================] - 8s 97ms/step - loss: 3.0960e-04 - val_loss: 3.6720e-04
Epoch 9/10
81/81 [==============================] - 8s 104ms/step - loss: 